<a href="https://colab.research.google.com/github/DrDAN6770/DataSciense-with-Python/blob/main/%E8%B3%87%E6%96%99%E7%A7%91%E5%AD%B8%E5%B7%A5%E5%85%B7%E7%AE%B1/%E6%95%99%E7%B7%B4%E5%B8%B6%E8%B7%91/Task%2003/Task_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#作業01

---


Yahoo 奇摩電影提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態。請試著撰寫爬蟲程式將 Yahoo 奇摩電影中的資訊抓下來，讓你快速瀏覽比較即將上映的電影有哪些值得關注的資訊。

❏ Sample Input: https://movies.yahoo.com.tw/movie_thisweek.html（資料來源：Yahoo 奇摩電影 ）

❏ Sample Output: 找出該頁面上的「中、英文名稱」、「上映日期」和「期待度」

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = 'https://movies.yahoo.com.tw/movie_thisweek.html'
r = requests.get(url)
response = r.text

In [ ]:
d = BeautifulSoup(response, 'html5lib')
movie_list = d.find_all('div',class_ = 'release_info')

In [ ]:
# 利用 id & class 限縮條件
movies_dict = {}
for movie in movie_list:
  movie_dict = {}
  ch_movie_name = movie.find('a', class_="gabtn").text.strip()
  en = movie.find('div',class_ = 'en').text.strip()
  release_movie_time = movie.find('div',class_='release_movie_time').text.split()[1]
  leveltext = movie.find('div',class_ = 'leveltext').text.strip().split('\n')[0]

  movie_dict['中文名稱'] = ch_movie_name
  movie_dict['英文名稱'] = en
  movie_dict['上映日期'] = release_movie_time
  movie_dict['期待度'] = leveltext
  movies_dict[movie_list.index(movie)+1] = movie_dict

  # print(f'中文名稱:{ch_movie_name};英文名稱:{en};上映日期:{release_movie_time};期待度:{leveltext}')
# movies_dict

In [ ]:
movie_dataframe = pd.DataFrame(movies_dict)
movie_dataframe.T

,中文名稱,英文名稱,上映日期,期待度
1,鬼玩人：復活,Evil Dead Rise,2023-04-20,100%
2,愛是自私,Egoist,2023-04-21,89%
3,月之圓缺,Phases of the Moon,2023-04-21,100%
4,電影刀劍亂舞-黎明-,Touken Ranbu: The Movie 2,2023-04-21,100%
5,炸彈客,The Bomber,2023-04-21,75%
6,屍智症,The Elderly,2023-04-21,100%
7,電影版 佐佐木與宮野 卒業篇,,2023-04-21,100%
8,寶可噩夢,Beau Is Afraid,2023-04-21,75%
9,詭樓,Lockdown Tower,2023-04-21,100%
10,絕地營救,The Covenant,2023-04-21,100%


#作業 02

---

我們發現在前一個練習中取得的資料只有 10 筆，也就是 Yahoo 奇摩電影「本週新片」頁面上呈現的資料。根據觀察後發現除了這 10 之外，還有更多資料會以分頁的方式呈現。除此之外，也能夠從「上映中」與「即將上映」的頁面中找到更多的電影類型。接下來這個練習，會希望你能夠除的單頁面的資料也可以收集到更多的資料回來。

❏ Sample Input: https://movies.yahoo.com.tw/movie_thisweek.html（資料來源：Yahoo 奇摩電影 ）

❏ Sample Output: 找出 Yahoo 奇摩電影中「本週新片」、「上映中」與「即將上映」前三個分頁中的電影，包含「中、英文名稱」、「上映日期」和「期待度」資訊。

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url_thisweek = 'https://movies.yahoo.com.tw/movie_thisweek.html'
url_intheaters = 'https://movies.yahoo.com.tw/movie_intheaters.html'
url_movie_comingsoon = 'https://movies.yahoo.com.tw/movie_comingsoon.html'
movie_info_bytime = [url_thisweek,url_intheaters,url_movie_comingsoon]

#分別讀取本週新片、上映中與即將上映
def bytime_movie_info(movie_info_bytime):
  DF = []
  for i in movie_info_bytime:
    res = get_movie_info(i)
    tempdf = pd.DataFrame(res)

    DF.append(tempdf)
  return DF

#讀取同一個項目中的前三頁
def get_movie_info(url,c = 3):
  #三頁的資料彙集一個字典
  all_movie_page = {}
  count = 0

  #每頁的資料
  for page in range(c):
    newurl = f'{url}?page={page+1}'
    r = requests.get(newurl)
    response = r.text

    d = BeautifulSoup(response, 'html5lib')
    movie_list = d.find_all('div',class_ = 'release_info')

    movies_dict = {}

    #每部電影的資料
    for movie in movie_list:
      count += 1
      movie_dict = {}
      ch_movie_name = movie.find('a', class_="gabtn").text.strip()
      en = movie.find('div',class_ = 'en').text.strip()
      release_movie_time = movie.find('div',class_='release_movie_time').text.split()[1]
      leveltext = movie.find('div',class_ = 'leveltext').find('span').text if movie.find('div',class_ = 'leveltext') else None

      movie_dict['中文名稱'] = ch_movie_name
      movie_dict['英文名稱'] = en
      movie_dict['上映日期'] = release_movie_time
      movie_dict['期待度'] = leveltext

      movies_dict[count] = movie_dict
    all_movie_page.update(movies_dict)
  return all_movie_page

res = bytime_movie_info(movie_info_bytime)

In [ ]:
#分成三大列(本週新片、上映中、即將上映)
# pd.concat(res,axis=1,keys=['本週新片','上映中','即將上映']).T #76*4

#分成三大行
pd.concat(res,axis=0,keys=['本週新片','上映中','即將上映']).T #30*12

本週新片                                               \
              中文名稱                       英文名稱        上映日期   期待度   
1           鬼玩人：復活             Evil Dead Rise  2023-04-20  100%   
2             愛是自私                     Egoist  2023-04-21   89%   
3             月之圓缺         Phases of the Moon  2023-04-21  100%   
4       電影刀劍亂舞-黎明-  Touken Ranbu: The Movie 2  2023-04-21  100%   
5              炸彈客                 The Bomber  2023-04-21   75%   
6              屍智症                The Elderly  2023-04-21  100%   
7   電影版 佐佐木與宮野 卒業篇                             2023-04-21  100%   
8             寶可噩夢             Beau Is Afraid  2023-04-21   75%   
9               詭樓             Lockdown Tower  2023-04-21  100%   
10            絕地營救               The Covenant  2023-04-21  100%   
11         不能流淚的悲傷        Behind The Blueeyes  2023-04-21   75%   
12          地中海發燒友        Mediterranean Fever  2023-04-21  100%   
13         阿凡達：水之道   Avatar: The Way of Water  2023-04-21  100%   
14            惡靈地堡            The Bunker Game  2023-04-21  100%   
15            烈火亞當                 Young Adam  2023-04-21   96%   
16              空巢                      Alone  2023-04-21  100%   
17             NaN                        NaN         NaN   NaN   
18             NaN                        NaN         NaN   NaN   
19             NaN                        NaN         NaN   NaN   
20             NaN                        NaN         NaN   NaN   
21             NaN                        NaN         NaN   NaN   
22             NaN                        NaN         NaN   NaN   
23             NaN                        NaN         NaN   NaN   
24             NaN                        NaN         NaN   NaN   
25             NaN                        NaN         NaN   NaN   
26             NaN                        NaN         NaN   NaN   
27             NaN                        NaN         NaN   NaN   
28             NaN                        NaN         NaN   NaN   
29             NaN                        NaN         NaN   NaN   
30             NaN                        NaN         NaN   NaN   

              上映中                                                         \
             中文名稱                                       英文名稱        上映日期   
1       吸血鬼特助：雷菲爾                                   Renfield  2023-04-14   
2        柴可夫斯基的妻子                         Tchaikovsky's Wife  2023-04-14   
3              疫起                           Eye of the Storm  2023-04-14   
4             涅槃咒                                Blood Karma  2023-04-14   
5             正年輕                              Forever Young  2023-04-14   
6            燈火闌珊                     A Light Never Goes Out  2023-04-14   
7              血忌                               Blood Flower  2023-04-14   
8      聖母院大火：世紀浩劫                         Notre Dame on Fire  2023-04-14   
9   穆荷蘭大道 4K數位修復版                           Mulholland Drive  2023-04-14   
10        假面騎士劇場版                              Battle Royale  2023-04-14   
11             厄林                          It’s In The Woods  2023-04-14   
12   小淘氣尼古拉：快樂的源頭            LITTLE NICHOLAS：Happy As Can Be  2023-04-14   
13           長春事件                             Eternal Spring  2023-04-07   
14             百花                          A Hundred Flowers  2023-04-07   
15        再一次勇敢跳躍                                       Rise  2023-04-07   
16    鬼玩人 4K數位修復版                               Evil Dead II  2023-04-07   
17        美麗的他電影版                 My Beautiful Man -eternal-  2023-04-07   
18          唐璜羅曼死                                   Don Juan  2023-04-07   
19             熊男                                   Bear Man  2023-04-07   
20     超級瑪利歐兄弟電影版                The Super Mario Bros. Movie  2023-04-05   
21            AIR                                        AIR  2023-04-05   
22           白日青春               The Sunny Side of the Street  2023-03-31   
23        65：恐怖行星                            

#作業03

---

在前述的練習中，你已經利用爬蟲成功的收集到了來自 Yahoo 奇摩電影中「本週新片」、「上映中」與「即將上映」前三個分頁中的電影，包含「中、英文名稱」、「上映日期」和「期待度」資訊。接下來請你將這些資料整理成一個 DataFrame 並且將資料存到到 SQLite 資料庫（名稱欄位自訂）中。

In [ ]:
dftoSQL = pd.concat(res,axis=0,keys=['本週新片','上映中','即將上映']).T #30*12

In [ ]:
# 存入SQL
import sqlite3

# 創SQL database
conn = sqlite3.connect('movie_info.db')

# 由不同的category創立table
for key in ['本週新片', '上映中', '即將上映']:
  dftoSQL[key].to_sql(key, con=conn,if_exists='replace',index=False)

conn.close()

In [ ]:
# 取出並彙整
res_from_sql = []
category = ['本週新片', '上映中', '即將上映']

conn = sqlite3.connect('movie_info.db')

# 選不同table建立dataframe再加入list(有NAN值會去除)
for key in category:
  res_from_sql.append(pd.read_sql_query(f'SELECT * FROM {key}', conn).dropna())

conn.close()

#show
dfres = pd.concat(res_from_sql,axis=1,keys=category)
dfres

本週新片                                               \
              中文名稱                       英文名稱        上映日期   期待度   
0           鬼玩人：復活             Evil Dead Rise  2023-04-20  100%   
1             愛是自私                     Egoist  2023-04-21   89%   
2             月之圓缺         Phases of the Moon  2023-04-21  100%   
3       電影刀劍亂舞-黎明-  Touken Ranbu: The Movie 2  2023-04-21  100%   
4              炸彈客                 The Bomber  2023-04-21   75%   
5              屍智症                The Elderly  2023-04-21  100%   
6   電影版 佐佐木與宮野 卒業篇                             2023-04-21  100%   
7             寶可噩夢             Beau Is Afraid  2023-04-21   75%   
8               詭樓             Lockdown Tower  2023-04-21  100%   
9             絕地營救               The Covenant  2023-04-21  100%   
10         不能流淚的悲傷        Behind The Blueeyes  2023-04-21   75%   
11          地中海發燒友        Mediterranean Fever  2023-04-21  100%   
12         阿凡達：水之道   Avatar: The Way of Water  2023-04-21  100%   
13            惡靈地堡            The Bunker Game  2023-04-21  100%   
14            烈火亞當                 Young Adam  2023-04-21   96%   
15              空巢                      Alone  2023-04-21  100%   
16             NaN                        NaN         NaN   NaN   
17             NaN                        NaN         NaN   NaN   
18             NaN                        NaN         NaN   NaN   
19             NaN                        NaN         NaN   NaN   
20             NaN                        NaN         NaN   NaN   
21             NaN                        NaN         NaN   NaN   
22             NaN                        NaN         NaN   NaN   
23             NaN                        NaN         NaN   NaN   
24             NaN                        NaN         NaN   NaN   
25             NaN                        NaN         NaN   NaN   
26             NaN                        NaN         NaN   NaN   
27             NaN                        NaN         NaN   NaN   
28             NaN                        NaN         NaN   NaN   
29             NaN                        NaN         NaN   NaN   

              上映中                                                         \
             中文名稱                                       英文名稱        上映日期   
0       吸血鬼特助：雷菲爾                                   Renfield  2023-04-14   
1        柴可夫斯基的妻子                         Tchaikovsky's Wife  2023-04-14   
2              疫起                           Eye of the Storm  2023-04-14   
3             涅槃咒                                Blood Karma  2023-04-14   
4             正年輕                              Forever Young  2023-04-14   
5            燈火闌珊                     A Light Never Goes Out  2023-04-14   
6              血忌                               Blood Flower  2023-04-14   
7      聖母院大火：世紀浩劫                         Notre Dame on Fire  2023-04-14   
8   穆荷蘭大道 4K數位修復版                           Mulholland Drive  2023-04-14   
9         假面騎士劇場版                              Battle Royale  2023-04-14   
10             厄林                          It’s In The Woods  2023-04-14   
11   小淘氣尼古拉：快樂的源頭            LITTLE NICHOLAS：Happy As Can Be  2023-04-14   
12           長春事件                             Eternal Spring  2023-04-07   
13             百花                          A Hundred Flowers  2023-04-07   
14        再一次勇敢跳躍                                       Rise  2023-04-07   
15    鬼玩人 4K數位修復版                               Evil Dead II  2023-04-07   
16        美麗的他電影版                 My Beautiful Man -eternal-  2023-04-07   
17          唐璜羅曼死                                   Don Juan  2023-04-07   
18             熊男                                   Bear Man  2023-04-07   
19     超級瑪利歐兄弟電影版                The Super Mario Bros. Movie  2023-04-05   
20            AIR                                        AIR  2023-04-05   
21           白日青春               The Sunny Side of the Street  2023-03-31   
22        65：恐怖行星                            